# When do visibility of artifacts and collections change?

Files and collections in LaminDB have the following 3 levels if visibility:
- 1: "default", visible in the UI by default, returned from the query results
- 0: "hidden", excluded from the query results, notebook artifacts (generated by `lamin save`) are default to be hidden
- -1: "trash", excluded from the query results, set with `.delete()`

## Setup

Install the `lamindb` Python package:
```shell
pip install lamindb
```

In [ ]:
!lamin init --storage test-visibility

In [ ]:
import lamindb as ln
import pandas as pd

In [ ]:
artifact = ln.Artifact.from_df(pd.DataFrame({"a": [1, 2], "b": [3, 4]}), description="mydf")
artifact.save()

New artifact has default visibility 1:

In [ ]:
assert artifact.visibility == 1

When you delete an artifact, its visibility is set to -1 ("trash"):

In [ ]:
artifact.delete()

In [ ]:
assert artifact.visibility == -1

Files in trash won't be returned from default queries:

In [ ]:
ln.Artifact.filter(description="mydf").all()

Unless you specify `visibility=None` to see all hidden and trashed artifacts:

In [ ]:
ln.Artifact.filter(description="mydf", visibility=None).all()

You can restore an artifact from trash:

In [ ]:
artifact.restore()

In [ ]:
assert artifact.visibility == 1

In [ ]:
ln.Artifact.filter(description="mydf").all()

Delete test artifact and instance:

In [ ]:
artifact.delete(permanent=True)

In [ ]:
!lamin delete --force test-visibility